In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from extract.locm import AP, Hypothesis, LOCM
from traces import *
from observation import ActionObservation
from extract.model import *

In [24]:
def get_example_obs_bw_2traces(print_trace=False):
    objects = {
        "A": PlanningObject("unknown", "A"),
        "B": PlanningObject("unknown", "B"),
        "C": PlanningObject("unknown", "C"),
    }

    actions = {
        "unstackAB": Action("unstack", [objects["A"], objects["B"]]),
        "unstackBC": Action("unstack", [objects["B"], objects["C"]]),
        "unstackCB": Action("unstack", [objects["C"], objects["B"]]),
        "unstackBA": Action("unstack", [objects["B"], objects["A"]]),
        "unstackAC": Action("unstack", [objects["A"], objects["C"]]),
        "unstackCA": Action("unstack", [objects["C"], objects["A"]]),
        "stackAB": Action("stack", [objects["A"], objects["B"]]),
        "stackBA": Action("stack", [objects["B"], objects["A"]]),
        "stackAC": Action("stack", [objects["A"], objects["C"]]),
        "stackCA": Action("stack", [objects["C"], objects["A"]]),
        "stackBC": Action("stack", [objects["B"], objects["C"]]),
        "stackCB": Action("stack", [objects["C"], objects["B"]]),
        "putdownA": Action("putdown", [objects["A"]]),
        "putdownB": Action("putdown", [objects["B"]]),
        "putdownC": Action("putdown", [objects["C"]]),
        "pickupA": Action("pickup", [objects["A"]]),
        "pickupB": Action("pickup", [objects["B"]]),
        "pickupC": Action("pickup", [objects["C"]]),
    }

    input_action_seqs = '''
        unstack(A, B), putdown(A), pickup(B), stack(B,C), unstack(B,C), stack(B,C),pickup(A), stack(A,B)
        unstack(C, B), putdown(C), unstack(B,A), putdown(B), pickup(B), stack(B,C), pickup(A), stack(A,B)
        unstack(C,B), stack(C,A), pickup(B), putdown(B), pickup(B), stack(B,C)
        '''
       
    traces = TraceList(
        [
            Trace(
                [
                    Step(State({}), actions["unstackAB"], 1),
                    Step(State({}), actions["putdownA"], 2),
                    Step(State({}), actions["pickupB"], 3),
                    Step(State({}), actions["stackBC"], 4),
                    Step(State({}), actions["unstackBC"], 5),
                    Step(State({}), actions["stackBC"], 6),
                    Step(State({}), actions["pickupA"], 7),
                    Step(State({}), actions["stackAB"], 8),
                ]
            ),
            Trace(
                [
                    Step(State({}),actions['unstackCB'], 1),
                    Step(State({}),actions['putdownC'], 2),
                    Step(State({}),actions['unstackBA'], 3),
                    Step(State({}),actions['putdownB'], 4),
                    Step(State({}),actions['pickupB'], 5),
                    Step(State({}),actions['stackBC'], 6),
                    Step(State({}),actions['pickupA'], 7),
                    Step(State({}),actions['stackAB'], 8),

                ]
            ),
            Trace(
                [
                    Step(State({}), actions["unstackCB"], 1),
                    Step(State({}), actions["stackCA"], 2),
                    Step(State({}), actions["pickupB"], 3),
                    Step(State({}), actions["putdownB"], 4),
                    Step(State({}), actions["pickupB"], 5),
                    Step(State({}), actions["stackBC"], 6),
                ]
            ),
            
        ]
    )
    

    if print_trace:
        # traces.print()
        traces.print("detail")

    obs = traces.tokenize(ActionObservation)
    return obs

In [25]:
obs_tracelist = get_example_obs_bw_2traces(True)

/home/uto/anu/codes/xLOCM/src/traces/trace_list.py:150: UserWarning: Invalid view detail. Defaulting to "details".
  warn(f'Invalid view {view}. Defaulting to "details".')


                                                       Trace                                                       
Attributes:                                                                                                        
  8 steps                                                                                                          
  0 fluents                                                                                                        
                                                       Steps                                                       
                     Step       State        Action                                                                
                        1                    unstack unknown A unknown B                                           
                        2                    putdown unknown A                                                     
                        3                    pickup unknown B                                                      
                        4                    stack unknown B unknown C                                             
                        5                    unstack unknown B unknown C                                           
                        6                    stack unknown B unknown C                                             
                        7                    pickup unknown A                                                      
                        8                    stack unknown A unknown B

                                                       Trace                                                       
Attributes:                                                                                                        
  8 steps                                                                                                          
  0 fluents                                                                                                        
                                                       Steps                                                       
                     Step       State        Action                                                                
                        1                    unstack unknown C unknown B                                           
                        2                    putdown unknown C                                                     
                        3                    unstack unknown B unknown A                                           
                        4                    putdown unknown B                                                     
                        5                    pickup unknown B                                                      
                        6                    stack unknown B unknown C                                             
                        7                    pickup unknown A                                                      
                        8                    stack unknown A unknown B

                                                       Trace                                                       
Attributes:                                                                                                        
  6 steps                                                                                                          
  0 fluents                                                                                                        
                                                       Steps                                                       
                     Step       State        Action                                                                
                        1                    unstack unknown C unknown B                                           
                        2                    stack unknown C unknown A                                             
                        3                    pickup unknown B                                                      
                        4                    putdown unknown B                                                     
                        5                    pickup unknown B                                                      
                        6                    stack unknown B unknown C

In [26]:
sorts = LOCM._get_sorts(obs_tracelist[0])

In [27]:
TS, ap_state_pointers, OS = LOCM._step1(obs_tracelist[0], sorts)

In [28]:
HS = LOCM._step3(TS,ap_state_pointers, OS, sorts)

In [29]:
bindings = LOCM._step4(HS)

In [30]:
bidings  = LOCM._step5(HS, bindings)

In [31]:
fluents, actions = LOCM._step7(OS, ap_state_pointers, sorts,bindings, {}, debug=True)


ap state pointers
{0: {stack.0: (7 -> 8),
     unstack.0: (1 -> 2),
     pickup.0: (5 -> 6),
     putdown.0: (3 -> 4)},
 1: {unstack.2: (1 -> 2),
     unstack.1: (1 -> 2),
     putdown.1: (3 -> 4),
     stack.1: (7 -> 8),
     stack.2: (3 -> 4),
     pickup.1: (5 -> 6)}}

OS:
{0: [{8, 1, 4, 5}, {2, 3, 6, 7}],
 1: [{1, 2, 3, 4, 5, 6, 7, 8}, {1}, {2}, {3}, {4}]}

bindings:
{}

{(sort0_state1 sort0),
 (sort1_state0 sort1),
 (sort0_state0 sort0),
 TemplateFluent(name='sort1_state2',
                param_sorts=['sort1']),
 TemplateFluent(name='sort1_state1',
                param_sorts=['sort1']),
 TemplateFluent(name='sort1_state4',
                param_sorts=['sort1']),
 TemplateFluent(name='sort1_state3',
                param_sorts=['sort1'])}
{(unstack sort0 sort1 sort1),
 (putdown sort0 sort1),
 (pickup sort0 sort1),
 (stack sort0 sort1 sort1)}


In [32]:
model = Model(fluents, actions)
assert model


In [33]:
model.to_pddl('locm')

ValueError: Could not determine whether the model is grounded or lifted. Fluents are of type <class 'extract.locm.LOCM._step7.<locals>.TemplateFluent'> while actions are of type <class 'extract.learned_action.LearnedLiftedAction'>